In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

df = pd.read_excel('City-Specific Data/Houston.xlsx')
df = df.dropna()

temperature = df['Temperature'].values.reshape(-1, 1)
energy_demand = df['EnergyDemand'].values.reshape(-1, 1)

scaler_temp = MinMaxScaler()
scaler_energy = MinMaxScaler()
temperature_normalized = scaler_temp.fit_transform(temperature)
energy_demand_normalized = scaler_energy.fit_transform(energy_demand)

def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target = data[i+seq_length]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

seq_length = 24
X, y = create_sequences(temperature_normalized, seq_length)
y = energy_demand_normalized[seq_length:]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

mse_scores = []
mae_scores = []
mape_scores = []

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_indices = y_true != 0
    return np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = Sequential([
        LSTM(50, activation='relu', input_shape=(seq_length, 1)),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.1, verbose=1)

    loss = model.evaluate(X_test, y_test)
    print(f"Fold Test loss: {loss}")

    predictions_normalized = model.predict(X_test)

    predictions = scaler_energy.inverse_transform(predictions_normalized)
    y_test_original = scaler_energy.inverse_transform(y_test)

    mse = mean_squared_error(y_test_original, predictions)
    mae = mean_absolute_error(y_test_original, predictions)
    mape = mean_absolute_percentage_error(y_test_original, predictions)
    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape)
    print(f"Fold Mean Squared Error: {mse}")
    print(f"Fold Mean Absolute Error: {mae}")
    print(f"Fold Mean Absolute Percentage Error: {mape}")

average_mse = np.mean(mse_scores)
average_mae = np.mean(mae_scores)
average_mape = np.mean(mape_scores)
print(f"Average Mean Squared Error: {average_mse}")
print(f"Average Mean Absolute Error: {average_mae}")
print(f"Average Mean Absolute Percentage Error: {average_mape}")

plt.figure(figsize=(12, 6))
plt.plot(scaler_energy.inverse_transform(y_test[:100]), label='Actual')
plt.plot(predictions[:100], label='Predicted')
plt.title('Predicted vs Actual Energy Demand for Houston (Last Fold)')
plt.xlabel('Time')
plt.ylabel('Energy Demand')
plt.legend()
plt.show()


Epoch 1/50


C:\Users\rushi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


998/998 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.0199 - val_loss: 0.0100
Epoch 2/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.0068 - val_loss: 0.0072
Epoch 3/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0062 - val_loss: 0.0071
Epoch 4/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0059 - val_loss: 0.0086
Epoch 5/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0058 - val_loss: 0.0072
Epoch 6/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0054 - val_loss: 0.0070
Epoch 7/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0054 - val_loss: 0.0066
Epoch 8/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0055 - val_loss: 0.0069
Epoch 9/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0053 - val_loss: 0.0068
Epoch 10/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0051 - val_loss: 0.0052
Epoch 11/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0050 - val_loss: 0.0063
Epoch 12/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.

C:\Users\rushi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


998/998 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 0.0303 - val_loss: 0.0068
Epoch 2/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0078 - val_loss: 0.0078
Epoch 3/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0074 - val_loss: 0.0105
Epoch 4/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0072 - val_loss: 0.0098
Epoch 5/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0066 - val_loss: 0.0072
Epoch 6/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0063 - val_loss: 0.0098
Epoch 7/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0060 - val_loss: 0.0066
Epoch 8/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.0058 - val_loss: 0.0090
Epoch 9/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.0056 - val_loss: 0.0060
Epoch 10/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.0054 - val_loss: 0.0064
Epoch 11/50
998/998 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.0053 - val_loss: 0.0083
Epoch 12/50
825/998 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.